---
title: "Data Cleaning"
format:
    html: 
        code-fold: false
---

The data cleaning process for this project involved several key steps aimed at preparing the raw food waste and food data for analysis. Initially, we removed unnecessary columns and filtered the data based on relevant criteria, such as valid serving size units. We also handled missing data by dropping rows with missing values in critical fields, ensuring the integrity of the dataset.

Next, we standardized measurement units by converting various food quantities to grams, ensuring consistency across the dataset. This was particularly important for nutritional analysis, where accurate comparisons are required. Additionally, we transformed specific columns (e.g., vitamins, cholesterol) into consistent units, further improving data uniformity.

We applied fuzzy matching techniques to merge food waste data with cleaned food data based on food name similarities. The fuzzy join method, using difflib’s SequenceMatcher, allowed us to handle slight variations in food names between datasets and find the best matches.

Throughout the process, we focused on ensuring that the data was in a format suitable for analysis, with correctly formatted data types, consistent units, and minimal missing or inconsistent data. This rigorous cleaning approach enables more reliable exploratory data analysis (EDA) and model building, setting the foundation for insightful analysis of food waste and nutrition.

## Managing Missing Data

•	Identify Missing Values: Missing values are common in real-world datasets and can occur for various reasons. In this project, missing data was identified in columns such as serving_size, protein, fat, carbs, calories, and other nutritional values. Using pandas’ isnull() function, the locations of missing values were pinpointed across the dataset.

•	Handling Missing Data: Several strategies were employed to handle missing data. For numerical columns like protein, fat, and calories, missing values were replaced with the mean of the column to retain the data’s overall distribution. In some cases, rows with missing critical data, such as serving_size, were dropped entirely to ensure the integrity of further analysis.

## Outlier Detection and Treatment

•	Identify Outliers: Outliers were detected using statistical methods such as Z-scores and box plots. Variables like serving_size, protein, fat, and other nutritional values were checked for extreme values that might indicate outliers.

•	Addressing Outliers: Outliers were addressed depending on their nature. In some cases, extreme values were capped or transformed, while in others, they were removed if they were clearly erroneous. Retaining outliers for analysis was also considered when they provided valuable insights into the data distribution.

•	Visualize Outliers: Box plots were used to visualize the outliers before and after handling. These plots illustrated the distribution of values and how outliers were managed in various columns. By comparing the before and after visualizations, the effect of outlier treatment on the dataset could be clearly seen.

## Normalization and Scaling:

•	Data Distribution Analysis: The distribution of numerical variables was initially analyzed using histograms and summary statistics to identify any skewness or irregular distributions. For example, variables like protein, carbs, and fat exhibited skewed distributions.

•	Normalization Techniques: To address this, normalization techniques such as min-max scaling and Z-score normalization were applied to certain columns. These methods ensured that variables with different ranges could be compared directly and improved the performance of certain machine learning models.

•	Before-and-After Visualizations: Before and after visualizations, such as histograms and box plots, were used to compare the data distributions. The effect of normalization could be seen in the flattening of skewed distributions, making them more suitable for modeling.


## Subsetting the Data

•	Data Filtering: The data was subsetted to focus on relevant columns and remove irrelevant ones. For example, columns related to allergens, microbes, and country of origin were removed, as they were not directly relevant to the analysis of food waste and nutritional content. Filtering also involved focusing on specific food categories and nutritional attributes, such as protein, fat, calories, and fiber.

•	Rationale: The rationale for filtering was to reduce the complexity of the data and focus on the most relevant features for analysis. By working with a smaller, more targeted subset, it was easier to perform focused analysis and avoid noise from unnecessary variables. The subsetted data was better suited for modeling and understanding the relationships between food waste and nutritional content.

## Code


In [ ]:
import requests
import pandas as pd
import json
import fitz  
import pdfplumber
from rapidfuzz import process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
from difflib import SequenceMatcher

food_data = pd.read_csv("data/raw-data/foods_data.csv")
food_waste = pd.read_csv("data/raw-data/ReFED_US_Food_Surplus_Detail.csv")

food_waste.drop(
    ['Unnamed: 0','sector', 'sub_sector', 'sub_sector_category',
       'food_type','surplus_upstream_100_year_mtco2e_footprint',
     'surplus_downstream_100_year_mtco2e_footprint',
     'surplus_total_100_year_mtco2e_footprint',
     'surplus_upstream_100_year_mtch4_footprint',
     'surplus_downstream_100_year_mtch4_footprint',
     'surplus_total_100_year_mtch4_footprint'], 
    axis=1, 
    inplace=True
)

food_data.drop(
    ['microbes', 'allergens', 'additives', 'labels',
       'nutrient_group', 'brand_name', 'food_labels', 'package_size',
       'food_safety_info', 'expiration_date', 'country_of_origin','fdc_id' ],
    axis=1, 
    inplace=True
)



valid_units = ['g', 'grm', 'gm', 'mlt', 'ml', 'mg']


df_filtered = food_data[food_data['serving_size_unit'].str.lower().isin(valid_units)]

df_filtered.columns

def convert_to_grams(row, column):
    unit = row['serving_size_unit'].lower()
    
    # If unit is grams (g, grm, gm), return the value as is
    if unit in ['g', 'grm', 'gm']:
        return row[column]
    # If unit is milliliters (ml, mlt), assume 1 ml = 1 g (for liquids)
    elif unit in ['ml', 'mlt']:
        return row[column]  # Convert 1 ml to 1 g
    # If unit is milligrams (mg), convert to grams
    elif unit == 'mg':
        return row[column] * 0.001  # 1 mg = 0.001 g
    # If unit is unknown or unsupported, return the value as is
    else:
        return row[column]

# List of columns to convert
columns_to_convert = [
    'serving_size', 'protein', 'percent_daily_value', 'fat', 'carbs', 'calories', 
    'fiber'
]

# Apply conversion to each specified column
for column in columns_to_convert:
    df_filtered[column] = df_filtered.apply(lambda row: convert_to_grams(row, column), axis=1)

def convert_to_grams(row):
    # Convert Vitamin A (IU to grams)
    vitamin_a_grams = row['vitamin_a_iu'] * 0.0000003 if pd.notnull(row['vitamin_a_iu']) else None
    
    # Convert Vitamin C (mg to grams)
    vitamin_c_grams = row['vitamin_c_mg'] * 0.001 if pd.notnull(row['vitamin_c_mg']) else None
    
    # Convert Cholesterol (mg to grams)
    cholesterol_grams = row['cholesterol_mg'] * 0.001 if pd.notnull(row['cholesterol_mg']) else None
    
    return pd.Series({'vitamin_a_grams': vitamin_a_grams, 'vitamin_c_grams': vitamin_c_grams, 'cholesterol_grams': cholesterol_grams})

# Apply the conversion function to the DataFrame
df_filtered[['vitamin_a_grams', 'vitamin_c_grams', 'cholesterol_grams']] = df_filtered.apply(convert_to_grams, axis=1)

# Drop the original columns
df_filtered = df_filtered.drop(columns=['vitamin_a_iu', 'vitamin_c_mg', 'cholesterol_mg'])

df_filtered = df_filtered.drop(columns=['brand', 'food_category', 'market_country'])

df_filtered.columns
df_filtered.head()

def mean_excluding_nulls(series):
    return series.dropna().mean()


df_grouped = df_filtered.groupby('food_name').agg({
    'serving_size': mean_excluding_nulls,
    'protein': mean_excluding_nulls,
    'percent_daily_value': mean_excluding_nulls,
    'fat': mean_excluding_nulls,
    'carbs': mean_excluding_nulls,
    'calories': mean_excluding_nulls,
    'fiber': mean_excluding_nulls,
    'calcium': mean_excluding_nulls,
    'iron': mean_excluding_nulls,
    'potassium': mean_excluding_nulls,
    'sodium': mean_excluding_nulls,
    'phosphorus': mean_excluding_nulls
}).reset_index()

In [ ]:

def calculate_overlap(str1, str2):
    matcher = SequenceMatcher(None, str1.lower(), str2.lower())
    return matcher.ratio()  
def find_best_match(food_name, df, used_matches):
    best_match = None
    highest_overlap = 0
    best_index = -1
    
    
    for idx, name2 in df['food_name'].items():  
        if idx not in used_matches:
            overlap = calculate_overlap(food_name, name2)
            if overlap > highest_overlap:
                best_match = name2
                highest_overlap = overlap
                best_index = idx
    
    return best_match, highest_overlap, best_index
matches = []
used_matches = set()

for name1 in food_waste['food_name']:
    best_match, score, best_index = find_best_match(name1, df_grouped, used_matches)
    if best_match and score >= 0.60:  
        matches.append({
            'food_name_food_waste': name1, 
            'best_match': best_match, 
            'score': score
        })
        used_matches.add(best_index) 

matches_df = pd.DataFrame(matches)

food_waste_renamed = food_waste.rename(columns={'food_name': 'food_name_food_waste'})

result = pd.merge(food_waste_renamed, matches_df, left_on='food_name_food_waste', right_on='food_name_food_waste', how='left')


result = pd.merge(result, df_grouped, left_on='best_match', right_on='food_name', how='left', suffixes=('_food_waste', '_grouped_df'))

print(result)
columns_to_convert = ['calcium', 'iron', 'potassium', 'sodium', 'phosphorus']

#convert to grams
for col in columns_to_convert:
    if col in result.columns:
        result[col] = result[col] / 1000

#drop rows where serving_size is NA
result = result.dropna(subset=['serving_size'])
result.columns
result.describe()

result.to_csv('data/processed-data/food_merged.csv')